In [2]:
import pandas as pd
import numpy as np
from PIL import Image
from PIL import ImageOps
import PIL
import torch, torchvision
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import math
import random
import os

In [2]:
def create_circular_mask(h, w, center=None, radius=None):

    if center is None: # use the middle of the image
        center = [int(w/2), int(h/2)]
    if radius is None: # use the smallest distance between the center and image walls
        radius = min(center[0], center[1], w-center[0], h-center[1])

    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

    mask = dist_from_center <= radius
    mask = mask.astype(int)
    return mask
mask = create_circular_mask(128,128)

In [3]:
img_path = '/home/rliu/ansim/data/data/JPEGImages/'
img_list_csv = '/home/rliu/github/ansim/img_list.csv'
train_csv = '/home/rliu/github/ansim/train.csv'
test_csv = '/home/rliu/github/ansim/test.csv'
img_list = pd.read_csv(img_list_csv)
seq_list = pd.read_csv(train_csv)
img_name = os.path.join(img_path, img_list.iloc[1, 0])
image = Image.open(img_name)
image = image.convert('L')
angle = 360 * np.random.uniform(0, 1)
image_resized = torchvision.transforms.functional.resize(image, (128,128), interpolation=2)
image_resized = torchvision.transforms.functional.rotate(image_resized, angle, resample=False, expand=False, center=None)
image_resized = image_resized * mask
image_tensor = torch.from_numpy(image_resized)

In [9]:
image_tensor.shape

torch.Size([128, 128])

In [6]:
image_resized

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [4]:
class ansimDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self,img_list_csv, seq_csv, root_dir, step=20, random_rotate = True, mask = mask, transform=None):
        """
        Args:
            image_csv (string): Path to the csv file with image path.
            seq_csv (string): Path to the csv file with indices of heads of sequence.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.img_list = pd.read_csv(img_list_csv)
        self.seq_list = pd.read_csv(seq_csv)
        self.root_dir = root_dir
        self.transform = transform
        self.step = step
        self.random_rotate = random_rotate
        self.mask = mask

    def __len__(self):
        return len(self.seq_list)

    def __getitem__(self, idx):
        seq_head = self.seq_list.iloc[idx,0]
        seq = torch.empty(self.step, 1, 128,128, dtype=torch.float)
        angle = 360 * np.random.uniform(0, 1)
        for i in np.arange(self.step):
            img_idx = seq_head + i
            img_name = os.path.join(self.root_dir, self.img_list.iloc[img_idx, 0])
            image = Image.open(img_name)
            image = image.convert('L')
            image_resized = torchvision.transforms.functional.resize(image, (128,128), interpolation=2)
            if self.random_rotate:
                image_resized = torchvision.transforms.functional.rotate(image_resized, angle, resample=False, expand=False, center=None)
            image_resized = image_resized * self.mask
            image_tensor = torch.from_numpy(image_resized)
            seq[i][0] = image_tensor
        if self.transform:
            seq = self.transform(seq)
        return seq

In [5]:
data_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.3019],
                             std=[0.1909])
    ])

In [6]:
img_path = '/home/rliu/ansim/data/data/JPEGImages/'
img_list_csv = '/home/rliu/github/ansim/img_list.csv'
train_csv = '/home/rliu/github/ansim/train.csv'
test_csv = '/home/rliu/github/ansim/test.csv'
ansim_dataset = ansimDataset(img_list_csv = img_list_csv, seq_csv = train_csv, root_dir = img_path, step=20, random_rotate = True, mask = mask, transform=None)
dataset_loader = torch.utils.data.DataLoader(ansim_dataset,
                                             batch_size=8, shuffle=True,
                                             num_workers=4)

In [7]:
dataiter = iter(dataset_loader)
images = dataiter.next()

In [29]:
images.shape[1]

20

10

In [32]:
(inputs, target) = torch.split(images, int(images.shape[1]/2), dim=1)

In [23]:
inputs.shape[1]

10

In [9]:
def imshow(img):
    img = img * 0.1909 + 0.3019     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

In [10]:
dataiter = iter(dataset_loader)
images, labels = dataiter.next()
imshow(torchvision.utils.make_grid(images))
classes = ["pos","neg","pos_o","nuc","non"]
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(8)))
print(labels)

ValueError: too many values to unpack (expected 2)

In [3]:
train = np.zeros(565)
test  = np.zeros(151)

In [4]:
for i in np.arange(185):
    train[i] = 112001 + 20*i
    
for i in np.arange(40):
    train[185+i] = 116471 + 20*i
    
for i in np.arange(170):
    train[185+40+i] = 117471 + 20*i
    
for i in np.arange(170):
    train[185+40+170+i] = 121471 + 20*i

In [5]:
for i in np.arange(51):
    test[i] = 115705 + 30*i
    
for i in np.arange(20):
    test[51+i] = 117171 + 30*i
    
for i in np.arange(40):
    test[51+20+i] = 120871 + 30*i
    
for i in np.arange(40):
    test[51+20+40+i] = 124871 + 30*i

In [6]:
df_train = pd.DataFrame(train)
df_train = df_train.astype(int)
df_test = pd.DataFrame(test)
df_test = df_test.astype(int)

In [7]:
df_train[0].to_csv('train200.csv',header=False,index=False)
df_test.to_csv('test200.csv',header=False,index=False)

In [8]:
df_train

,0
0,112001
1,112021
2,112041
3,112061
4,112081
5,112101
6,112121
7,112141
8,112161
9,112181


In [9]:
df_img_list = pd.DataFrame([["%0.6d.jpg" % (x+1)] for x in range(125470)], columns=['image path'])
df_img_list.to_csv('img_list.csv',header=False,index=False)

In [ ]:
df_img_list.head(10)

In [ ]:
np.arange()